<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [2]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [3]:
"""
Import Statements
"""

# Base
from collections import Counter
import re
 
import pandas as pd

# Plotting
import squarify
import matplotlib.pyplot as plt
import seaborn as sns

# NLP Libraries
import spacy
from spacy.tokenizer import Tokenizer
from nltk.stem import PorterStemmer


nlp = spacy.load("en_core_web_lg")

In [4]:
# clean the dataset

from bs4 import BeautifulSoup

def clean_text(text):
    text = BeautifulSoup(text, "html.parser").get_text() #removes html via beautiful soup library
    text = text.replace('\\n', ' ') #replaces newline with spaces
    text = text.replace('/', ' ') #replaces backslashes withs spaces
    text = text.lower() #makes all text lower case
    text = re.sub(r'[^a-zA-Z ^0-9]', '', text) #makes just letters and numbers
    text = re.sub(r'(x.[0-9])', '', text) #takes out special characters
    return text

yelp['text'] = yelp.apply(lambda x: clean_text(x['text']), axis=1)

In [5]:
yelp['text'].head(15)

0     beware fake fake fakewe also own a small busin...
1     came here for lunch togo service was quick sta...
2     ive been to vegas dozens of times and had neve...
3     we went here on a night where they closed off ...
4     35 to 4 starsnot bad for the price 1299 for lu...
5     tasty fast casual latin street food  the menu ...
6     this show is absolutely amazing what an incred...
7     came for the pho and really enjoyed it  we got...
8     absolutely the most unique experience in a nai...
9     wow i walked in and sat at the bar for 10 minu...
10    we popped in for dinner yesterday with no rese...
11    thw worst stay ever so first i ended up paying...
12    great friendly customer service and quality fo...
13    the food was great  it was super busy but our ...
14    talk about getting ripped off they charged us ...
Name: text, dtype: object

In [6]:
#intial the tokenizer
nlp = spacy.load("en_core_web_lg")
tokenizer = Tokenizer(nlp.vocab)
STOP_WORDS = nlp.Defaults.stop_words

In [7]:
#tokenizer pipe - lemmatize and remove stops/blanks
tokens = []

for doc in tokenizer.pipe(yelp['text']):
    doc_tokens = []
    for token in doc:
        if (token.lemma_ not in STOP_WORDS) & (token.text != ' '):
            doc_tokens.append(token.lemma_)
    tokens.append(doc_tokens)
    
yelp['tokens'] = tokens
yelp['tokens'].head()

0    [beware, fake, fake, fakewe, small, business, ...
1    [come, lunch, togo, service, quick, staff, fri...
2    [ive, vega, dozen, time, step, foot, circus, c...
3    [night, close, street, party, actually, group,...
4    [35, 4, starsnot, bad, price, 1299, lunch, sen...
Name: tokens, dtype: object

In [8]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,tokens
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,beware fake fake fakewe also own a small busin...,10,n1LM36qNg4rqGXIcvVXv8w,"[beware, fake, fake, fakewe, small, business, ..."
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,came here for lunch togo service was quick sta...,0,5CgjjDAic2-FAvCtiHpytA,"[come, lunch, togo, service, quick, staff, fri..."
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,ive been to vegas dozens of times and had neve...,2,BdV-cf3LScmb8kZ7iiBcMA,"[ive, vega, dozen, time, step, foot, circus, c..."
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,we went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ,"[night, close, street, party, actually, group,..."
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,35 to 4 starsnot bad for the price 1299 for lu...,5,n9QO4ClYAS7h9fpQwa5bhA,"[35, 4, starsnot, bad, price, 1299, lunch, sen..."


## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [9]:
def tokenize(document):
    doc = nlp(document)
    return [token.lemma_.strip() for token in doc if (token.is_stop != True) and (token.is_punct != True) and (token.text != ' ')]

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# create the transformer
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words=STOP_WORDS)

# build vocab
tfidf.fit(yelp['text'])

# transform text
dtm = tfidf.transform(yelp['text'])
print(dtm.shape)

### if we have test set we just fit transform on data set, and just transform on real set

# Create a Vocabulary -- once we run we have built vocabulary
# The vocabulary establishes all of the possible words that we might use.
# The vocabulary dictionary does not represent the counts of words!!


/Users/blakelobato/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
/Users/blakelobato/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['d', 'm', 've'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(10000, 31976)


In [12]:
dtmwc = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())
dtmwc.head()

,,-PRON-,0,001695do,001695howard,007,01,011802,02,025,...,zuma,zumanity,zumba,zuni,zupas,zuzana,zuzu,zyrtec,zzaplon,zzzzzzzzzmy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
from sklearn.neighbors import NearestNeighbors

# Fit on DTM
nn = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')
nn.fit(dtm)


/Users/blakelobato/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/neighbors/_base.py:414: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [20]:
dumbie_rev = ["""Vegas is the most fun city in the world."""]
new = tfidf.transform(dumbie_rev)
nn.kneighbors(new.todense())

(array([[1.        , 1.        , 1.21691924, 1.22769062, 1.23143705,
         1.24681331, 1.25493332, 1.25565358, 1.26381685, 1.26397793]]),
 array([[6204, 6311,  450, 1251, 6566, 4905, 5526, 4251, 1768, 9974]]))

In [21]:
yelp['text'][6311]

''

In [22]:
yelp['text'][6204]

' '

In [23]:
yelp['text'][450]

'i been coming here since i move here from new york city five years ago from new york city theres no better other place than this barbershop here they no they cuts and they know how to make a good line for you if you need it'

In [24]:
yelp['text'][1251]

'my favorite place in the world friendly employees starbucks and shopping what more could you ask for'

In [25]:
yelp['text'][6566]

'they finally got it right  good price point  this city has been missing something like this for a long time  give it a try'

In [26]:
yelp['text'][4905]

'fabulous fun small place with excellent bar and fantastic menu salads dogs and sandwiches are all good atmosphere is cozy friendly and fun'

In [27]:
yelp['text'][5526]

'we love coming here the combination of two worlds is awesome the place is fantastic the service is pretty good but the food is amazing we just love coming here'

In [28]:
yelp['text'][4251]

'absolutely out of this world  the rolls are the best ive had anywhere  our favorite sushi restaurant by a long shot  friendly staff  cant wait to go back'

In [29]:
yelp['text'][1768]

'this is my favorite hotel in the world it has a good feeling like home and the fountains are amazing it is very clean and very comfortable'

In [30]:
yelp['text'][9974]

'i had almost given up on finding a good mexican restaurant in vegas until i stumbled upon this amazing place first we were given delicious basket of chips salsa and beans before ordering we ordered the fajitas and it came with a ridiculous amount of yummy sides the roasted jalapeos were out of this world and so was the fideo soup that came with the entre you have to check this place out best mexican food in vegas'

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [31]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

In [32]:
clf = rfc

pipe = Pipeline([('vect', vect), ('clf', clf)])

In [34]:
parameters = {
    'vect__min_df': (0.02, 0.05),
    'vect__max_df': (0.75, 1.0),
    'vect__max_features': (500, 1000),
    'clf__max_depth':(5,10,15,20),
    'clf__n_estimators': (25, 100)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
revs = yelp['text'].tolist()
target = yelp['stars'].to_numpy()
grid_search.fit(revs, target)

Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   41.6s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  3.6min
[Parallel(n_jobs=4)]: Done 320 out of 320 | elapsed:  7.1min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [38]:
print(grid_search.best_score_)
grid_search.predict(dumbie_rev)

#5 star on a good response saying most fun ever

0.5524


array([5])

In [39]:
bad_rev = ["""I HATE THIS PLACE WORST EVER"""]
grid_search.predict(bad_rev)
#1 star on a bad review makiing sense with the model

array([1])

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [43]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary
import gensim

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora


In [45]:
review_total = yelp.shape[0]
revs = []

for i in range (0, review_total):
    revs.append(tokenize(yelp['text'][i]))

Learn the vocubalary of the yelp data:

In [46]:
# A Dictionary Representation of all the words in our corpus
id2word = corpora.Dictionary(revs)

In [48]:
id2word.token2id['vegas']

117

In [49]:
id2word.doc2bow(tokenize("This is a sample review about how cool Vegas is."))

[(260, 1), (557, 1), (716, 1)]

In [50]:
import sys
print(sys.getsizeof(id2word))
print(sys.getsizeof(tokens))

56
87624


In [51]:
len(id2word.keys())

32039

In [52]:
# Let's remove extreme values from the dataset
id2word.filter_extremes(no_below=5, no_above=0.95)

In [53]:
len(id2word.keys())
#awesome cut extreme fat went from 32,039 to 6088!!!

6088

In [55]:
corpus = [id2word.doc2bow(text) for text in revs]

Create a bag of words representation of the entire corpus

In [57]:
corpus[117][:10]

[(8, 2),
 (27, 1),
 (44, 1),
 (89, 2),
 (272, 1),
 (289, 1),
 (364, 1),
 (436, 1),
 (474, 2),
 (494, 2)]

Your LDA model should be ready for estimation: 

In [59]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   iterations=5,
                   workers=4,
                   num_topics = 10 # You can change this parameter
                  )

Create 1-2 visualizations of the results

In [60]:
lda.print_topics()

[(0,
  '0.018*"not" + 0.012*"place" + 0.011*"food" + 0.011*"good" + 0.010*"come" + 0.010*"service" + 0.009*"order" + 0.009*"great" + 0.009*"like" + 0.008*"go"'),
 (1,
  '0.016*"not" + 0.014*"food" + 0.013*"place" + 0.010*"time" + 0.010*"good" + 0.008*"come" + 0.008*"order" + 0.007*"great" + 0.007*"go" + 0.007*"get"'),
 (2,
  '0.021*"not" + 0.012*"place" + 0.011*"time" + 0.011*"good" + 0.011*"like" + 0.010*"come" + 0.009*"service" + 0.009*"great" + 0.008*"food" + 0.008*"order"'),
 (3,
  '0.015*"not" + 0.013*"good" + 0.011*"food" + 0.010*"place" + 0.010*"time" + 0.008*"great" + 0.008*"service" + 0.007*"order" + 0.007*"try" + 0.007*"like"'),
 (4,
  '0.014*"not" + 0.013*"good" + 0.009*"great" + 0.008*"like" + 0.008*"time" + 0.008*"go" + 0.008*"come" + 0.007*"food" + 0.007*"service" + 0.006*"get"'),
 (5,
  '0.017*"not" + 0.012*"good" + 0.011*"great" + 0.010*"food" + 0.010*"place" + 0.009*"order" + 0.008*"time" + 0.008*"service" + 0.008*"get" + 0.008*"come"'),
 (6,
  '0.017*"good" + 0.013*"n

In [61]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]
topics = [' '.join(t[0:5]) for t in words]

for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
not place food good come

------ Topic 1 ------
not food place time good

------ Topic 2 ------
not place time good like

------ Topic 3 ------
not good food place time

------ Topic 4 ------
not good great like time

------ Topic 5 ------
not good great food place

------ Topic 6 ------
good not come food time

------ Topic 7 ------
not place good great food

------ Topic 8 ------
good not place great time

------ Topic 9 ------
good not place food great



In [62]:
from gensim.models.coherencemodel import CoherenceModel

def compute_coherence_values(dictionary, corpus, limit, start=2, step=3, passes=5):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    limit : Max num of topics
    passes: the number of times the entire lda model & coherence values are calculated

    Returns:
    -------
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    coherence_values = []
    
    for iter_ in range(passes):
        for num_topics in range(start, limit, step):
            model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, workers=4)
            coherencemodel = CoherenceModel(model=model,dictionary=dictionary,corpus=corpus, coherence='u_mass')
            coherence_values.append({'pass': iter_, 
                                     'num_topics': num_topics, 
                                     'coherence_score': coherencemodel.get_coherence()
                                    })

    return coherence_values

In [ ]:
# Can take a long time to run.
coherence_values = compute_coherence_values(dictionary=id2word, 
                                                        corpus=corpus,
                                                        start=2, 
                                                        limit=40, 
                                                        step=2,
                                                        passes=100)

In [ ]:
topic_coherence = pd.DataFrame.from_records(coherence_values)
topic_coherence.head()

In [ ]:
import seaborn as sns

ax = sns.lineplot(x="num_topics", y="coherence_score", data=topic_coherence)

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)